# munging.py

In [20]:
import pandas as pd
a = pd.DataFrame({'id': ['a', 'b', 'c'], 2: ['x', 'y', 'z']})
b = pd.DataFrame({'id': ['a', 'b', 'c'], 'sort': ['j', 'l', 'k']})
print(a, b)

  id  2
0  a  x
1  b  y
2  c  z   id sort
0  a    j
1  b    l
2  c    k


In [16]:
sort = b.sort_values('sort')['id']
sort_map = dict(zip(sort, range(len(sort))))
sort_map

{'a': 0, 'c': 1, 'b': 2}

In [19]:
a['sort_temp'] = a['id'].map(sort_map)
a = a.sort_values('sort_temp')
a = a.drop('sort_temp', axis=1)
a

,id,2
0,a,x
2,c,z
1,b,y


In [23]:
%%writefile ../j_utils/munging.py
# %load munging.py
import numpy as np
import pandas as pd
from pandas.api.types import is_string_dtype, is_numeric_dtype
import re
from tqdm import tqdm


# Training Utils?
def time_series_data_split(eva, date_col, q, n_yields):
    '''
    passing in sorted eva df and date_col, returns train and valid,
    val_frac as valid data size. uses panda qcut behind the scenes
    returns n_yields number of sets of indicies
    '''
#     step = int(val_frac * len(eva))
    # get date in date_col at split, return everything
    eva = eva.copy()
    eva['temp'] = pd.qcut(eva['issue_d'], q=q, labels=range(q))
    counter = 0
    for i in range(q-1, q-1-n_yields, -1):
        val_idx = eva.query('temp == @i').index
        counter -= len(val_idx)
        tr_idx = eva.index[:counter]
        yield tr_idx, val_idx


def sort_train_eval(train, eva, id_col, sort_col, assert_shape=True):
    '''
    Helps to sort train df by sort_col which is only in eva df. train and
    eva should both contain id_col (the unique identifier).
    assert_shape checks that every id_col in train and eva are one-to-one
    '''
    if assert_shape:
        train_id = set(train[id_col])
        eva_id = set(eva[id_col])
        assert not train.duplicated(id_col).any(), print('first df has id dupes')
        assert not eva.duplicated(id_col).any(), print('second df has id dupes')
        assert train_id == eva_id, print('some ids not present in both dfs')
        assert train.shape[0] == eva.shape[0], print('ids not one to one')
    eva = eva.sort_values(sort_col)
    sorted_id = eva[id_col]
    sort_map = dict(zip(sorted_id, range(len(sorted_id))))
    train['sort_temp'] = train[id_col].map(sort_map)
    train = train.sort_values('sort_temp')
    train = train.drop('sort_temp', axis=1)
    return train, eva
    

def check_train_valid(train, valid, id_col, original=None):
    '''
    Function to check that two df's (train, valid) do not contain the same
    data samples based on id_col. If passing original full set, it checks
    that all samples are accounted for as well (if splitting train and valid
    from original)
    '''
    train_id = set(train[id_col])
    test_id = set(valid[id_col])
    assert len(train_id.intersection(test_id)) == 0
    if original:
        assert train_id.union(test_id) == set(original[id_col])
        print('all ids accounted for')
    print('no overlapping id cols found')

# MUNGING ____________________________________________________________________
def train_proc(df, normalize = True, verbose=True, isnull=True):
    '''
    get_noninf_val, replace_infs with respective max or min value
    get_normalize_info, normalize_df (if needed)
    transform dates
    make_null_ind_cols
    get_fill_values, fill_values
    get_categories, encode_categories
    remove_zerovar_cols
    
    Returns list of:
    df, all_train_colnames, max_dict, min_dict, new_null_colnames, fill_dict, cats_dict, (norm_dict)
    '''

    max_dict, min_dict = get_noninf_val(df)
    replace_infs(df, max_dict, min_dict)
    if normalize:
        norm_dict = get_normalize_info(df)
        normalize_df(df, norm_dict)
    transform_dates(df)
    if isnull:
        df, new_null_colnames = make_null_ind_cols(df)
    fill_dict = get_fill_values(df)
    fill_values(df, fill_dict)
    cats_dict = get_categories(df)
    encode_categories(df, cats_dict)
    remove_zerovar_cols(df, verbose=verbose)
    to_ret = [df, df.columns, max_dict, min_dict, fill_dict, cats_dict]
    if isnull:
        to_ret.append(new_null_colnames)
    if normalize:
        to_ret.append(norm_dict)
    return to_ret

def val_test_proc(df, all_train_colnames, max_dict, min_dict, fill_dict, cats_dict, norm_dict={}, verbose=True, isnull = True):
    '''
    # Make cols match
    # replace_infs
    # noramlize_df (if needed)
    # transform dates
    # make_null_ind_cols
    # fill_values
    # encode_categories
    # select cols to match train
    '''
    for col in all_train_colnames:
        if col not in df.columns:
            df[col] = np.nan
    
    transform_dates(df)
    replace_infs(df, max_dict, min_dict)
    if norm_dict:
        normalize_df(df, norm_dict)
    if isnull:
        df, new_null_colnames = make_null_ind_cols(df)
    fill_values(df, fill_dict)
    encode_categories(df, cats_dict)
    df = df[all_train_colnames]

    return df
# sub_functions

def add_datepart(df, fldname, drop=True, time=False):
    """add_datepart converts a column of df from a datetime64 to many columns containing
    the information from the date. This applies changes inplace.

    Parameters:
    -----------
    df: A pandas data frame. df gain several new columns.
    fldname: A string that is the name of the date column you wish to expand.
        If it is not a datetime64 series, it will be converted to one with pd.to_datetime.
    drop: If true then the original date column will be removed.
    time: If true time features: Hour, Minute, Second will be added.

    Examples:
    ---------

    >>> df = pd.DataFrame({ 'A' : pd.to_datetime(['3/11/2000', '3/12/2000', '3/13/2000'], infer_datetime_format=False) })
    >>> df

        A
    0   2000-03-11
    1   2000-03-12
    2   2000-03-13

    >>> add_datepart(df, 'A')
    >>> df

        AYear AMonth AWeek ADay ADayofweek ADayofyear AIs_month_end AIs_month_start AIs_quarter_end AIs_quarter_start AIs_year_end AIs_year_start AElapsed
    0   2000  3      10    11   5          71         False         False           False           False             False        False          952732800
    1   2000  3      10    12   6          72         False         False           False           False             False        False          952819200
    2   2000  3      11    13   0          73         False         False           False           False             False        False          952905600
    """
    fld = df[fldname]
    if not np.issubdtype(fld.dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second', 'Microsecond', 'Nanosecond']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
#     df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

def transform_dates(df):
    '''
    Looks for datetime columns in the df, uses fastai's add_datepart to turn it
    into several columns (year, day, is quarter end, etc.)
    Does this inplace.
    '''
    date_cols = list(df.select_dtypes('datetime').columns)
    for col in date_cols:
        add_datepart(df, col,)


def get_inf_cols(df):
    '''
    Gets colnames with inf in them
    '''
    num_cols = df.select_dtypes([np.number]).columns
    inf_cols = [col for col in num_cols if any(np.isinf(df[col]))]
    return inf_cols

def get_noninf_val(df):
    inf_cols = get_inf_cols(df)
    max_dict = df[inf_cols].replace([np.inf], np.nan).max().to_dict()
    min_dict = df[inf_cols].replace([-np.inf], np.nan).min().to_dict()
    for k,v in max_dict.items():
        max_dict[k] = v*100 #was inf so make it 100x larger than max
    for k,v in min_dict.items():
        if v > 0:
            v = v/100
        elif v <= 0:
            v = v*100
        min_dict[k] = v #was -inf so make it 100x larger than min    
    return max_dict, min_dict

def replace_infs(df, max_dict, min_dict):
    for col, val in max_dict.items():
        df[col] = df[col].replace([np.inf], val)
    for col, val in min_dict.items():
        df[col] = df[col].replace([-np.inf], val)
                
def get_null_cols(df):
    '''
    Gets null colnames in df if there are any
    '''
    null_cols = [col for col in df.columns if any(df[col].isnull())]
    return null_cols

def make_null_ind_cols(df):
    '''
    Had to return or else cols weren't actually added inplace
    '''
    null_cols = get_null_cols(df)
    for col in null_cols:
        df[col+'_isnull'] = np.where(df[col].isnull(), 1, 0)
    return df, [col+'_isnull' for col in null_cols]        
        
def get_fill_values(df):
    '''
    Only to be used on train. Get the right fill values for every col.
    '''
    assert len(df.columns) == len(df.select_dtypes([np.number, 'object', 'bool']).columns)
    all_cols = df.columns
    fill_dict = {}
    for col in all_cols:
        if is_numeric_dtype(df[col]):
            fill_val = df[col].median() # take median
            if not np.isfinite(fill_val):
                try:
                    fill_val = df[col].mode()[0] # take mode
                except IndexError:
                    fill_val = 0
                finally:
                    if not np.isfinite(fill_val):
                        fill_val = 0
                
        elif is_string_dtype(df[col]):
            try:
                fill_val = df[col].mode()[0] # take mode
            except IndexError:
                fill_val = '0'
            
        # if fill val still not finite, set to 0
#         if not np.isfinite(fill_val):
#             fill_val = 0 # if mode is nan or inf
        fill_dict[col] = fill_val
    return fill_dict

def fill_values(df, fill_dict):
#     assert set(df.columns) == set(fill_dict.keys())
    df.fillna(fill_dict, inplace=True)

def get_categories(df):
    '''
    only gets categories.
    '''
    obj_cols = df.select_dtypes('object')
    cats_dict = {}
    for col in obj_cols:
        cats_dict[col] = df[col].astype('category').cat.categories
    return cats_dict
            
def encode_categories(df, cats_dict):
    for col, cats in cats_dict.items():
        df[col] = pd.Categorical(df[col], categories = cats)
        df[col] = df[col].cat.codes + 1 #so nan -> -1 becomes 0
            
def remove_zerovar_cols(df, ret_cols=False, verbose=False):
    '''
    Iterates throught columns and checks nunique(). If nunique == 1, then the whole
    column has only one value, and will be dropped inplace. Prints out which
    columns it has dropped and can return them if desired.
    '''
    zerovar_cols = []
    consider_drop_cols = []
    for col in df.columns:
        nunique = df[col].nunique(dropna=False)
        if nunique <= 1:
            zerovar_cols.append(col)
        else:
            var = df[col].var()
            if (not np.isfinite(var)) | (var == 0):
                zerovar_cols.append(col)
    df.drop(zerovar_cols, axis=1, inplace=True)

            
    for col in df.columns:
        nunique = df[col].nunique(dropna=False)
        if nunique == 2:
            if '_isnull' not in col:
                consider_drop_cols.append(col)

    if verbose:
        print('dropping the following cols: \n{0}'.format(zerovar_cols))           
        print('only 2 values, consider dropping the following cols: \n{0}'.format(consider_drop_cols))
    if ret_cols:
        return zerovar_cols, consider_drop_cols        
        
def all_numeric(df):
    '''
    Returns True if the df is completely numeric.
    '''
    if df.shape[1] == df._get_numeric_data().shape[1]:
        return True
    else:
        return False

def get_normalize_info(df):
    num_df = df.select_dtypes(np.number)
    assert all_numeric(num_df)
    norm_dict = {}
    means = {}
    stds = {}
    rel_cols = [col for col in num_df.columns if '_isnull' not in col]
    std_notfin = {}
    for col in rel_cols:
        mean = num_df[col].mean()
        std = num_df[col].std()
        if np.isfinite(std):
            means[col] = mean
            stds[col] = std
        else:
            std_notfin[col] = std
    norm_dict['means'] = means
    norm_dict['stds'] = stds
    norm_dict['std_notfin'] = std_notfin
    return norm_dict

def normalize_df(df, norm_dict):
    means = norm_dict['means']
    stds = norm_dict['stds']
    for col in means.keys():
        mean = means[col]
        std = stds[col]
        df[col] = (df[col]-mean)/std
        
def reduce_memory(df, verbose=True):
    '''
    Tries to cast float and int dtype cols to smallest possible for dataframe.
    For saving RAM/disk space.
    '''
    if verbose:
        print('trying to change columns to smaller dtypes when possible')
        ori_mem = df.memory_usage(deep=True).sum()
        print('original dataframe is {0} MB or {1} GB'.format(ori_mem/(1024**2), ori_mem/(1024**3)))
    dict_to_df = {}
    changed_type_cols = []
    reducible = df.select_dtypes(['int', 'float'])
    irreducible = df[[col for col in df.columns if col not in reducible.columns]]
    for col in tqdm(reducible.columns):
        col_type = df[col].dtypes.name
        max_val = df[col].max()
        min_val = df[col].min()
        int_types = ['int32', 'int16', 'int8']
        float_types = ['float32'] #float 16 not supported in feather format?, 'float16']
        np_typedict = np.typeDict
        if 'float' in col_type:
            type_list = float_types
            infoer = np.finfo
        elif 'int' in col_type:
            type_list = int_types
            infoer = np.iinfo
        ok_dtypes = []
        for dtype in type_list:
            dt_max = infoer(dtype).max
            dt_min = infoer(dtype).min
            if (max_val <= dt_max) & (min_val >= dt_min):
                ok_dtypes.append(dtype)
        try:
            cast_dtype = ok_dtypes[-1]
        except IndexError:
            cast_dtype = col_type
        if cast_dtype != col_type:
            dict_to_df[col] = df[col].astype(cast_dtype)
            changed_type_cols.append(col)
    print('changed dtypes of {0} cols'.format(len(changed_type_cols)))
    reduced = pd.DataFrame(dict_to_df)
    smaller = pd.concat([irreducible, reduced], axis=1)        
    if verbose:
        small_mem = smaller.memory_usage(deep=True).sum()
        print('reduced dataframe is {0} MB or {1} GB'.format(small_mem/(1024**2), small_mem/(1024**3)))
    return changed_type_cols, smaller

Overwriting ../j_utils/munging.py


# file_manipulation.py

In [86]:
import os
import stat
from pwd import getpwnam, getpwuid
import shutil


def find_owner(filename):
    return getpwuid(os.stat(filename).st_uid).pw_name

def is_group_writeable(filepath):
    '''
    https://stackoverflow.com/questions/1861836/checking-file-permissions-in-linux-with-python
    '''
    st = os.stat(filepath)
    return bool(st.st_mode & stat.S_IWGRP)

def make_group_writeable(filepath):
    if not is_group_writeable(filepath):
        pass

In [69]:
dpath = '/home/justin/projects/lendingclub/data/'
find_owner(dpath + 'raw_loan_info.fth')

'jenkins'

In [87]:
ls {dpath}

csvs/  example.py  raw_loan_info.fth  raw_pmt_hist_1.fth


In [88]:
getpwnam('justin')

pwd.struct_passwd(pw_name='justin', pw_passwd='x', pw_uid=1000, pw_gid=1000, pw_gecos='Justin Hsi,,,', pw_dir='/home/justin', pw_shell='/bin/bash')

In [89]:
getpwnam('jenkins')

pwd.struct_passwd(pw_name='jenkins', pw_passwd='x', pw_uid=126, pw_gid=135, pw_gecos='Jenkins,,,', pw_dir='/var/lib/jenkins', pw_shell='/bin/bash')

In [97]:
print(os.getegid())

1000


In [105]:
shutil.chown(dpath + 'raw_pmt_hist_1.fth', user='justin', group='justin')

In [103]:
%debug

> /home/justin/anaconda3/lib/python3.6/shutil.py(1046)chown()
   1044             raise LookupError("no such group: {!r}".format(group))
   1045 
-> 1046     os.chown(path, _user, _group)
   1047 
   1048 def get_terminal_size(fallback=(80, 24)):

ipdb> _user
1000
ipdb> _group
135
ipdb> os.chown(path, _user, _group)
*** PermissionError: [Errno 1] Operation not permitted: '/home/justin/projects/lendingclub/data/raw_pmt_hist_1.fth'
ipdb> q


In [101]:
shutil.chown(dpath + 'raw_loan_info.fth', group='jenkins')

PermissionError: [Errno 1] Operation not permitted: '/home/justin/projects/lendingclub/data/raw_loan_info.fth'

In [84]:
os.chown(dpath + 'raw_pmt_hist_1.fth', 126, 135)

PermissionError: [Errno 1] Operation not permitted: '/home/justin/projects/lendingclub/data/raw_pmt_hist_1.fth'

In [80]:
os.chmod(dpath + 'raw_loan_info.fth', 0o664)

PermissionError: [Errno 1] Operation not permitted: '/home/justin/projects/lendingclub/data/raw_loan_info.fth'

In [75]:
os.chmod(dpath + 'raw_pmt_hist_1.fth', 0o664)

In [61]:
ls

Dockerfile*                Jenkinsfile*     README.md
eval_results.py*           models.py*       test_munging.py*
feature_pruning.py*        munging.py*      writing_the_modules.ipynb
hypothesis_example.ipynb*  other_utils.py*
__init__.py*               __pycache__/


In [70]:
os.chown??

# testing code

In [1]:
pd.set_option('max_columns', 500)

In [2]:
df = pd.read_feather(os.path.join(os.path.expanduser('~'), 'projects', 'lendingclub', 'data', 'raw_loan_info.fth'))

In [3]:
from munging import *

In [4]:
changed_type_cols, df = reduce_memory(df)

trying to change columns to smaller dtypes when possible
original dataframe is 5266.260987281799 MB or 5.142832995392382 GB


100%|██████████| 1/1 [00:00<00:00, 30.58it/s]


changed dtypes of 1 cols
reduced dataframe is 5257.195958137512 MB or 5.133980427868664 GB


In [5]:
ori_df = df.copy()
ori_df.shape

In [20]:
df = ori_df.copy()
df.shape

(2376343, 149)

In [23]:
num_cols = df.select_dtypes([np.number]).columns

In [38]:
num_cols

Index(['id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'annual_inc', 'dti', 'delinq_2yrs',
       ...
       'deferral_term', 'hardship_amount', 'hardship_length', 'hardship_dpd',
       'orig_projected_additional_accrued_interest',
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'settlement_amount', 'settlement_percentage', 'settlement_term'],
      dtype='object', length=113)

In [43]:
get_noninf_val(df[['recoveries', 'hardship_amount']].div(df['delinq_amnt'], axis=0))

({'recoveries': 151435.99853515625, 'hardship_amount': 1139.222240447998},
 {'recoveries': 0.0, 'hardship_amount': 2.692669630050659e-05})

In [14]:
df, max_dict, min_dict, new_null_colnames, fill_dict, cats_dict, norm_dict = train_proc(df)
df.describe()

dropping the following cols: 
['hardship_type', 'policy_code', 'deferral_term', 'hardship_length', 'policy_code_isnull', 'deferral_term_isnull', 'hardship_length_isnull']
only 2 values, consider dropping the following cols: 
['pymnt_plan', 'initial_list_status', 'application_type', 'hardship_flag', 'debt_settlement_flag', 'term']


,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,url,...,sec_app_collections_12_mths_ex_med_isnull,sec_app_mths_since_last_major_derog_isnull,hardship_amount_isnull,hardship_dpd_isnull,orig_projected_additional_accrued_interest_isnull,hardship_payoff_balance_amount_isnull,hardship_last_payment_amount_isnull,settlement_amount_isnull,settlement_percentage_isnull,settlement_term_isnull
count,2.376343e+06,2.376343e+06,2.376343e+06,2.376343e+06,2.376343e+06,2.376343e+06,2.376343e+06,2.376343e+06,2.376343e+06,2.376343e+06,...,2.376343e+06,2.376343e+06,2.376343e+06,2.376343e+06,2.376343e+06,2.376343e+06,2.376343e+06,2.376343e+06,2.376343e+06,2.376343e+06
mean,2.644243e+00,1.118249e+01,2.798029e+05,4.415229e+00,3.914463e+00,1.936336e+00,7.494740e+01,3.894764e+00,1.000205e+00,1.188172e+06,...,9.475236e-01,9.828127e-01,9.951640e-01,9.951640e-01,9.961862e-01,9.951640e-01,9.951640e-01,9.843171e-01,9.843171e-01,9.843171e-01
std,1.255025e+00,6.283129e+00,1.481650e+05,3.151404e+00,1.913116e+00,7.778774e-01,4.052204e+01,2.191074e+00,1.431415e-02,6.859913e+05,...,2.229858e-01,1.299690e-01,6.937303e-02,6.937303e-02,6.163846e-02,6.937303e-02,6.937303e-02,1.242456e-01,1.242456e-01,1.242456e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.000000e+00,6.000000e+00,1.507050e+05,2.000000e+00,2.000000e+00,1.000000e+00,4.200000e+01,2.000000e+00,1.000000e+00,5.940865e+05,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
50%,3.000000e+00,1.100000e+01,2.982590e+05,3.000000e+00,5.000000e+00,2.000000e+00,7.700000e+01,2.000000e+00,1.000000e+00,1.188172e+06,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
75%,3.000000e+00,1.500000e+01,4.181740e+05,6.000000e+00,6.000000e+00,3.000000e+00,1.130000e+02,6.000000e+00,1.000000e+00,1.782258e+06,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
max,7.000000e+00,3.500000e+01,5.268560e+05,1.100000e+01,6.000000e+00,3.000000e+00,1.420000e+02,9.000000e+00,2.000000e+00,2.376343e+06,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [44]:
norm_dict

{'means': {'id': 83586239.45349346,
  'loan_amnt': 15156.7783203125,
  'funded_amnt': 15151.6904296875,
  'funded_amnt_inv': 15134.6689453125,
  'term': 42.95010779167822,
  'int_rate': 13.033180236816406,
  'installment': 447.61492919921875,
  'annual_inc': 78292.375,
  'dti': 18.901901245117188,
  'delinq_2yrs': 0.3030197322368622,
  'fico_range_low': 694.5877685546875,
  'fico_range_high': 699.08203125,
  'inq_last_6mths': 0.5703238844871521,
  'mths_since_last_delinq': 34.630218505859375,
  'mths_since_last_record': 72.87171173095703,
  'open_acc': 11.617355346679688,
  'pub_rec': 0.1936313956975937,
  'revol_bal': 16713.69140625,
  'total_acc': 24.111675262451172,
  'out_prncp': 4289.98828125,
  'out_prncp_inv': 4289.1025390625,
  'total_pymnt': 12078.12109375,
  'total_pymnt_inv': 12060.4921875,
  'total_rec_prncp': 9518.5087890625,
  'total_rec_int': 2421.72998046875,
  'total_rec_late_fee': 1.5540074110031128,
  'recoveries': 148.2235870361328,
  'collection_recovery_fee': 24.8

In [16]:
min_dict

{}

In [11]:
proced_train = train_proc(ex_df)

dropping the following cols: 
['member_id', 'policy_code', 'hardship_type', 'deferral_term', 'hardship_length', 'member_id_isnull', 'policy_code_isnull', 'deferral_term_isnull', 'hardship_length_isnull']
only 2 values, consider dropping the following cols: 
['term', 'pymnt_plan', 'initial_list_status', 'application_type', 'hardship_flag', 'debt_settlement_flag']


# old

In [12]:


# # old
    
# # def get_medians_make_nullcols_fill_values(df, na_map = {}, catboost=False):
#     '''
#     This function makes new columns marking if a column is null or not, and
#     then fills the original column with median values. Does this inplace on
#     the passed dataframe, but still returns the dataframe as well. IF col 
#     dtype is not numeric, fills with mode instead. IF df[col].median() is 
#     nan, fills with 0
#     '''
#     if not na_map:
#         has_nulls = [col for col in df.columns if any(df[col].isnull())]
#         for col in tqdm(has_nulls):
#             if catboost:
#                 # catboost wanted nans in string dtypes to be string...
#                 if is_string_dtype(df[col]):
#                     df[col] = df[col].replace(np.nan, 'nan')
#                     na_map[col] = 'nan'

#             # normal stuff
#             if is_numeric_dtype(df[col]):
#                 if np.isnan(df[col].median()):
#                     # handles case where the whole col is nan and you don't want to drop
#                     # for reasons (old data doesn't have feature, but new data does)
#                     df[col] = df[col].fillna(0)
#                     na_map[col] = 0
#                 else:
#                     median = df[col].median()
#                     df[col] = df[col].fillna(median)
#                     na_map[col] = median
#             else:
#                 print('{0} col is likely of non-int/float dtype, filling with mode instead'.format(col))
#                 mode = df[col].mode() #is pandas series
#                 if len(mode) == 0:
#                     mode = 0
#                 else:
#                     mode = np.random.choice(mode) #if len(1), return mode, else ranodmly choose 1
#                 df[col] = df[col].fillna(mode)
#                 na_map[col] = mode
#         return df, na_map
#     else:
#         print('Was passed an na_map. Will fill accordingly (test or valid data?)')
#         for col, nafill in na_map.items():
#             df[col] = df[col].fillna(nafill)
#         return df

# # def prepare_nulls_valid_test(valid_test, train, 

# # def normalize_df(df, means_stds = {}):
# #     '''
# #     This function should be run after get_medians_make_nullcols_fill_values.
# #     Doesn't normalize the "_isnull" cols that are added by the aforemetioned
# #     function. Will print out a list of columns that it did not normalize but
# #     those cols should be checked if normalization is actually desired. 
# #     means_stds are for valid or test sets, normalized with same values as train
# #     '''
# #     if not means_stds:
# #         unsure_cols = []
# #         means = {}
# #         stds = {}
# #         for col in tqdm(df.columns):
# #             if '_isnull' not in col:
# #                 mean = df[col].mean()
# #                 means[col] = mean
# #                 std = df[col].std()
# #                 stds[col] = std
# #                 df[col] = (df[col]-mean)/std
# #             elif '_isnull' in col:
# #                 pass
# #             else:
# #                 if 'null' in col.lower() or 'is_null' in col.lower():
# #                     unsure_cols.append(col)
# #         print('These cols have word null or is_null in them. Double check. {0}'.format(
# #             unsure_cols))
# #         means_stds['means'] = means
# #         means_stds['stds'] = stds
# #         return df, means_stds
# #     else:
# #         print('Passed means_stds, normalizing cols according to means_stds (is valid or test set)')
# #         means = means_stds['means']
# #         stds = means_stds['stds']
# #         for col in means.keys():
# #             df[col] = (df[col] - means[col])/stds[col]
# #         return df
    
# # def jproc_df(df, target=None, one_hot=False, copy=True, summary=True): #broken out into functions below, probably deprecated?
#     '''
#     Should be run on df after:
#     1. mg.transform_dates, (turns datetime columns into ML usable)
#     2. mg.remove_zerovar_cols, and consider drop cols is examined
#     3. fastai's train_cats (turns obj/str cols into categorical)
    
#     This function will convert categoricals to their codes, adding +1 (so nan
#     is 0 instead of -1), and then will calculate means/stddev for
#     normalizing and median for filling nan values. Also creates new cols
#     demarkating whether value was originally nan via
#     mg.get_medians_make_nullcols_fill_values.
    
#     copy could be set to False if you suspect memory issues
    
#     returns x, y, na_dict, mapper
#     '''
    
#     if target:
#         y = df[target]
#     else:
#         print('No specified target column, assuming target already separated')
#         y = []
#     if copy:
#         df = df.copy()
    
#     # deal with cat cols, can either onehot or just turn into the categorical code
#     cat_cols = df.select_dtypes('category').columns
#     if one_hot:
#         print('Turning categoricals into one_hot representation')
#         dummied = pd.get_dummies(df[cat_cols])
#         df.drop(cat_cols, axis=1, inplace=True)
#     else:
#         print('Converting categoricals to their codes . . .')
#         for col in tqdm(cat_cols):
#             df[col] = df[col].cat.codes+1
    
#     # all other (numeric) cols
#     # gather the means/stddevs/nas __________________
#     print('Calculating means/medians/std_devs . . .')
#     all_other_cols = [col for col in df.columns if col not in cat_cols]
#     mapper = {}
#     na_dict = {}
#     for col in tqdm(all_other_cols):
#         mapper[col] = {'mean': df[col].mean(),
#                        'std_dev': df[col].std()}
#         na_dict[col] = df[col].median()
        
#     # make na cols, fill nas with median
#     print('Making _isnull indicator columns . . .')
#     has_nulls = [col for col in df.columns if any(df[col].isnull())]
#     for col in tqdm(has_nulls):
#         df[col+'_isnull'] = np.where(df[col].isnull(), 1, 0)
#         df[col] = df[col].fillna(na_dict[col])
        
#     # normalize the df excluding cat_cols
#     print('Normalizing all non-categorical and non-_isnull columns . . .')
#     all_other_cols = [col for col in df.columns if col not in cat_cols]
#     for col in tqdm(all_other_cols):
#         if '_isnull' not in col:
#             df[col] = (df[col]-mapper[col]['mean'])/mapper[col]['std_dev']
    
#     if one_hot:
#         df = pd.concat([df, dummied], axis=1)
    
#     if summary:
#         print('Categorical cols: {0}\n\n'.format(list(cat_cols)))
#         print('Made _isnull cols for: {0}\n\n'.format(list(has_nulls)))
#         print('Normalized all other cols: {0}\n\n'.format([col for col in all_other_cols if '_isnull' not in col]))
            
#     return df, y, na_dict, mapper